POSSIBLE PROBLEM DEFINITIONS TO WORK ON AS SUGGESTED BY TEAM MEMBERS

1. prediction of terrorist organization responsible for an attack

2. Prediction of country or continent susceptible to terrorist attack

3. Story telling/analysis of countries more affected with terrorism attack and recommendation system for international aide based on impact and occurrence of terrorist attack.

4. Designing a forecasting system for future terrorist attack based on trend of past attacks

5. Analyze, predict and categorize various terrorist activities

6. predicting success of a terrorist attack

7. Predicting region of a terrorist attack

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plb
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer

In [11]:
!pip install colorama

In [12]:
plt.style.use('seaborn')
from colorama import Style, Fore, Back

In [5]:
df = pd.read_csv("clean_df.csv", index_col=[0])

In [6]:
# function to display UNIQUE examples of data in each field

def getDesc(data, Column, numExamples=5):
    x = list(set([i for i in data[Column] if i==i]))
    if len(x)<numExamples:
        numExamples = len(x)
    x = list(np.random.choice(x, size=numExamples, replace=False))
    print('Examples in '+Style.BRIGHT+Back.CYAN+Column+Back.RESET+':'+Style.RESET_ALL, x, '\n')
    return None

# ======================================================================================

# function to display the number of missing values in each field

def getNumNan(data, thisFeature):
    ms = data[data[thisFeature] != data[thisFeature]].shape[0]
    print('Missing values in '+Style.BRIGHT+Back.CYAN+thisFeature+Back.RESET+Style.RESET_ALL+': {}'.format(ms))
    return None

In [13]:
for i in df.columns:
    getNumNan(df, i)
    getDesc(df, i, 7)

Missing values in Year: 0
Examples in Year: [2008, 1981, 1987, 2006, 1982, 1984, 1989] 

Missing values in Month: 0
Examples in Month: [12, 7, 0, 11, 9, 8, 5] 

Missing values in Day: 0
Examples in Day: [16, 9, 8, 11, 17, 25, 14] 

Missing values in Extended: 0
Examples in Extended: [0, 1] 

Missing values in Country: 0
Examples in Country: ['East Germany (GDR)', 'Laos', 'Sierra Leone', 'China', 'Luxembourg', 'India', 'Spain'] 

Missing values in Region: 0
Examples in Region: ['Southeast Asia', 'Australasia & Oceania', 'Middle East & North Africa', 'Central Asia', 'South America', 'Eastern Europe', 'Western Europe'] 

Missing values in City: 0
Examples in City: ['Ein Tamer', 'Fitadher', 'Dantar', 'Bhagirath Para', 'Cerro La  Caja', 'Gourma Rhaous', 'Bouberak Sahel'] 

Missing values in Success: 0
Examples in Success: [1, 0] 

Missing values in Suicide: 0
Examples in Suicide: [0, 1] 

Missing values in Attack_Type: 0
Examples in Attack_Type: ['Hijacking', 'Bombing/Explosion', 'Assassina

In [32]:
df['Region'].nunique()

12

In [14]:
# Function for the creation of the date column, and correction of wrong day or month
def make_date(df):
    new_date = ""
    year = df['Year']
    month = df['Month']
    day = df['Day']
    
    if month < 1:
        month = 1
        
    if day < 1:
        day = 1
    
    new_date = f"{year}-{month}-{day}"
    
    return new_date

In [15]:
df = df.assign(
    Date=df.apply(make_date, axis=1)
)
df.Date = pd.to_datetime(df.Date)

In [16]:
df.info() # to check the data type of the Date column

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181691 entries, 0 to 181690
Data columns (total 28 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Year                181691 non-null  int64         
 1   Month               181691 non-null  int64         
 2   Day                 181691 non-null  int64         
 3   Extended            181691 non-null  int64         
 4   Country             181691 non-null  object        
 5   Region              181691 non-null  object        
 6   City                181691 non-null  object        
 7   Success             181691 non-null  int64         
 8   Suicide             181691 non-null  int64         
 9   Attack_Type         181691 non-null  object        
 10  Target_Type         181691 non-null  object        
 11  Target_Subtype      181691 non-null  object        
 12  Target              181691 non-null  object        
 13  Target_Nationality  181691 no

In [17]:
df.drop(['Day', 'Month', 'Date'], axis=1, inplace=True)

In [19]:
#The following columns will be dropped from the predictors_df because it has object as it data type, it was done so as to rescale the dataset
independent_var_df = df.drop(columns = ['Attack_Group','Country','Region','City','Attack_Type','Target_Type','Target_Subtype','Target','Target_Nationality','Weapon','gname','target1'], axis=1)
independent_var_df.shape

(181691, 14)

In [21]:
independent_var_df = independent_var_df.drop(columns = ['Date'], axis=1)

In [22]:
#RESCALING OF THE DATASET
#Rescaling of the dataset was done due to the large differnce between the minimum and maximum values across the entire dataset
#For the rescaling the feature_range will be from 0 t0 1 
scaler = MinMaxScaler(feature_range=(0,1))
rescaled_df = pd.DataFrame(scaler.fit_transform(independent_var_df), columns= independent_var_df.columns)
rescaled_df.head()

,Year,Extended,Success,Suicide,No_Of_Killed,No_Of_Wounded,Property,country,region,attacktype1,targtype1,natlty1,weaptype1
0,0.0,0.0,1.0,0.0,0.000637,0.0,0.9,0.054,0.090909,0.000,0.619048,0.054,1.000000
1,0.0,0.0,1.0,0.0,0.000000,0.0,0.9,0.126,0.000000,0.625,0.285714,0.017,1.000000
2,0.0,0.0,1.0,0.0,0.000637,0.0,0.9,0.156,0.363636,0.000,0.428571,0.213,1.000000
3,0.0,0.0,1.0,0.0,0.000000,0.0,1.0,0.074,0.636364,0.250,0.285714,0.213,0.416667
4,0.0,0.0,1.0,0.0,0.000000,0.0,1.0,0.097,0.272727,0.750,0.285714,0.213,0.583333


In [23]:
#NORMALIZATION
#From the visualization of the distribution using histogram it can be seen that some features in the dataset were not normal
#Therefore normalization was done to remove skewness from the dataset
scaler = Normalizer()
normalized_df = pd.DataFrame(scaler.fit_transform(rescaled_df), columns= rescaled_df.columns)
normalized_df.head()

,Year,Extended,Success,Suicide,No_Of_Killed,No_Of_Wounded,Property,country,region,attacktype1,targtype1,natlty1,weaptype1
0,0.0,0.0,0.558379,0.0,0.000356,0.0,0.502541,0.030152,0.050762,0.000000,0.345663,0.030152,0.558379
1,0.0,0.0,0.550613,0.0,0.000000,0.0,0.495552,0.069377,0.000000,0.344133,0.157318,0.009360,0.550613
2,0.0,0.0,0.559401,0.0,0.000356,0.0,0.503461,0.087267,0.203418,0.000000,0.239743,0.119152,0.559401
3,0.0,0.0,0.600457,0.0,0.000000,0.0,0.600457,0.044434,0.382109,0.150114,0.171559,0.127897,0.250191
4,0.0,0.0,0.566723,0.0,0.000000,0.0,0.566723,0.054972,0.154561,0.425042,0.161921,0.120712,0.330588


In [36]:
df['Region'].unique()

array(['Central America & Caribbean', 'North America', 'Southeast Asia',
       'Western Europe', 'East Asia', 'South America', 'Eastern Europe',
       'Sub-Saharan Africa', 'Middle East & North Africa',
       'Australasia & Oceania', 'South Asia', 'Central Asia'],
      dtype=object)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [38]:
mapper = {'Central America & Caribbean': 0, 'North America' : 1, 'Southeast Asia' : 2, 'Western Europe' : 3, 'East Asia' : 4,  'South America': 5, 'Eastern Europe':6,
       'Sub-Saharan Africa' : 7 , 'Middle East & North Africa' : 8, 'Australasia & Oceania':9, 'South Asia': 10, 'Central Asia' : 11 }

df['Region'] = df['Region'].map(mapper)

In [40]:
df['Region'].dtype

dtype('int64')

In [41]:
X = normalized_df
y= df['Region']

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [43]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
#from sklearn.neural_network import MLPClassifier

clf = [ LogisticRegression(random_state=2), DecisionTreeClassifier(random_state=2), SVC (random_state=2),
       RandomForestClassifier(random_state=2), GradientBoostingClassifier(random_state=2) ]
models = [ 'Logistic Regression', 'Tree', 'Support vector machine', 'RFC', 'Gradient boost' ]



In [44]:
for clf, model in zip(clf,models):
  clf.fit ( X_train, y_train )
  y_pred = clf.predict ( X_test )
  print ( f'Cross validation score of {model}: %.3f \n' %cross_val_score (clf, X_train, y_train, cv=5).mean() )

Cross validation score of Logistic Regression: 0.795 

Cross validation score of Tree: 0.992 



KeyboardInterrupt: 